## Ground Control Stations

This notebook serves to get familiar with the simulator

In [9]:
from simulator import Simulator
from simulator.config import DATA_PATH, Color
from simulator.helpers import clean
from simulator.helpers.coordinates import ENUPose, GRAPose
from simulator.planner import AutoPlan, Plan
from simulator.visualizer import (
              QGC,
              Gazebo,
              GazMarker,
              NoVisualizer,
              QGCMarker,
              SimVehicle,
)

clean()

## Simulation Positions
 

In [10]:
gra_origin = GRAPose(lat=-35.3633280, lon=149.1652241,alt=0,heading=0) 
enu_origin = ENUPose(x=0, y=0, z=gra_origin.alt, heading=gra_origin.heading) 

base_homes= ENUPose.list([  # east, north, up, heading
    (5, 5, 0, 0),
    (10, 0, 0, 0),
    (-5, -10, 0, 0),
    (-15, 0, 0, 0),
    (0, -20, 0, 0),

])
base_paths = [Plan.create_square_path(side_len=slen, alt=5) for slen in (5, 7, 4, 3, 2)]

## Create Vehicles

In [11]:
sysids = [1,2,3,4,5]
colors = 2*[Color.BLUE]+3*[Color.GREEN]

vehs:list[SimVehicle] = []
for sysid, base_home, base_path, color in zip(sysids, base_homes, base_paths, colors):
    mission_path = DATA_PATH / f"mission_{sysid}.waypoints"
    plan = AutoPlan(
        name="simple_auto_plan",
        mission_path=str(mission_path),
    )

    plan.save_basic_mission_from_relative(
        sysid=sysid,
        gra_origin=gra_origin,
        relative_home=base_home,
        relative_path=base_path,
    )

    veh = SimVehicle.from_relative(
        sysid=sysid,
        gcs_name=f'{color.name}_{color.emoji}',
        plan=plan,
        color=color,
        enu_origin=enu_origin,
        relative_home=base_home,
        relative_path=base_path,
        model="iris",
    )
    vehs.append(veh)
    

## Visualizer

### Gazebo

In [12]:
gaz= Gazebo(gra_origin,world_path="simulator/visualizer/gazebo/worlds/runway.world")
origin_gaz = GazMarker(name="origin",
                    group="origin",
                    pos=enu_origin.unpose(),
                    color=Color.WHITE)
gaz.markers.append(origin_gaz)

### QGroundControl

In [13]:
qgc= QGC(gra_origin)
origin_qgc = QGCMarker(name="origin",
                pos=gra_origin.unpose(),
                color=Color.WHITE)
qgc.markers.append(origin_qgc)

### No Visualizer

In [14]:
novis = NoVisualizer(gra_origin)

## Simulator

In [15]:
simulator = Simulator(
	visualizer=gaz,
	terminals=['gcs'],
	verbose=1,
)
for veh in vehs:
    simulator.add_vehicle(veh)

simulator.show()

## Run

In [16]:
orac = simulator.launch()
orac.run()

00:43:11 - Oracle ⚪ - INFO - 🖥️  Gazebo launched for realistic simulation and 3D visualization.
00:43:11 - Oracle ⚪ - INFO - 🚀 GCS BLUE_🟦 launched (PID 1442986)
00:43:11 - Oracle ⚪ - INFO - 🚀 GCS GREEN_🟩 launched (PID 1442988)
00:43:11 - Oracle ⚪ - INFO - 🏁 Starting Oracle with 5 vehicles and 2 GCSs
00:44:30 - Oracle ⚪ - INFO - UAV 5 completed mission and exited
00:44:32 - Oracle ⚪ - INFO - UAV 4 completed mission and exited
00:44:36 - Oracle ⚪ - INFO - Received DONE from GCS GREEN_🟩
00:44:36 - Oracle ⚪ - INFO - UAV 3 completed mission and exited
00:44:37 - Oracle ⚪ - INFO - UAV 1 completed mission and exited
00:44:49 - Oracle ⚪ - INFO - Received DONE from GCS BLUE_🟦
00:44:49 - Oracle ⚪ - INFO - UAV 2 completed mission and exited
00:44:49 - Oracle ⚪ - INFO - ✅ All GCS threads completed
00:44:49 - Oracle ⚪ - INFO - 🎉 Oracle shutdown complete!
